In [1]:
from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.operators import OnePointCrossOver, GaussianMutation
from torch import Tensor
import random
import torch
import math

In [39]:
def distance(x, y):
    return math.sqrt((x[0]-y[0])**2+(x[1]-y[1])**2)


def fun(x):
    #Huge penalty for non unique indexes
    uniq = x.unique()
    if(uniq.size(0) != x.size(0)):
        return (x.size(0)-uniq.size(0))*penalty
    dist = distance(cityList[x[x.size(0) - 1]], cityList[x[0]])
    for i in range(x.size(0) - 1):
        dist += distance(cityList[x[i]], cityList[x[i + 1]])
    return dist


def mutation(values: torch.Tensor) -> torch.Tensor:
    u = values.unique()
    #Makes indexes all unique
    if(u.size(0) != values.size(1)):
        all = list(range(city_count))
        for i in u:
            all.remove(i)
        random.shuffle(all)
        values = torch.cat([u, torch.Tensor(all)])

    #Swaps values in population randomly
    swap_count = math.floor(city_count*0.2)
    for i in range(swap_count):
        r1 = random.randint(0, city_count-1)
        r2 = random.randint(0, city_count-2)
        if(r2 >= r1):
            r2 += 1
        values[r1], values[r2] = values[r2], values[r1]
    return values


def create_city_list(city_count: int) -> list:
    val_range = city_count*1000
    return [
        (random.randrange(val_range), random.randrange(val_range))
        for _ in range(city_count)
    ]

In [40]:
max_dist = 0
popsize = 100
city_count = 1000
cityList = create_city_list(city_count)

for i in cityList:
    for j in cityList:
        dist = distance(i, j)
        if(dist > max_dist):
            max_dist = dist
penalty = max_dist * len(cityList)

In [41]:
problem = Problem("min", fun, bounds=(0, city_count-1), solution_length=city_count, dtype=torch.int64)

searcher = GeneticAlgorithm(problem, popsize=popsize, operators=[
    OnePointCrossOver(problem, tournament_size=math.floor(city_count*0.1)),
    mutation
])

[2024-05-29 09:37:59] INFO     < 5237> evotorch.core: Instance of `Problem` (id:139710877360592) -- The `dtype` for the problem's decision variables is set as torch.int64
[2024-05-29 09:37:59] INFO     < 5237> evotorch.core: Instance of `Problem` (id:139710877360592) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2024-05-29 09:37:59] INFO     < 5237> evotorch.core: Instance of `Problem` (id:139710877360592) -- The `device` of the problem is set as cpu
[2024-05-29 09:37:59] INFO     < 5237> evotorch.core: Instance of `Problem` (id:139710877360592) -- The number of actors that will be allocated for parallelized evaluation is 0


In [44]:
for i in range(10):
    searcher.run(20)
    print(i, ": ", searcher.status["best_eval"])

0 :  210304843776.0
1 :  210304843776.0
2 :  210304843776.0
3 :  210304843776.0
4 :  210304843776.0
5 :  210304843776.0
6 :  210304843776.0
7 :  210304843776.0
8 :  210304843776.0
9 :  210304843776.0


In [43]:
searcher.status

<LazyStatusDict
    pop_best = <not yet computed>
    median_eval = <not yet computed>
    pop_best_eval = <not yet computed>
    mean_eval = <not yet computed>
    iter = 100
    best = <Solution values=tensor([  0,   2,   4,   5,   6,   7,   8,   9,  10,  12,  96,  14,  15,  16,
         17,  18,  19,  20,  22, 579,  24,  25,  26,  27,  29,  30,  31,  32,
         39,  35,  36,  37,  38,  39,  40,  42,  43,  44,  45, 578,  47,  48,
         49,  51,  52,  53,  54, 465,  57,  58,  59, 434, 625,  63,  64,  65,
        860,  67,  68, 309,  72,  73,  74,  75,  76,  77, 441, 711,  81, 433,
         84,  85,  86,  89,  90,  91,  92,  93,  94, 385,  96,  98, 794, 100,
        101, 102, 103, 104, 106, 107, 893, 112,  83, 115, 116, 117, 236, 119,
        159, 122, 123, 124, 125, 126, 128, 129, 131, 132, 133, 134, 135, 136,
        137, 139, 800, 141, 144, 199, 187, 149, 150, 152, 968, 154, 155, 158,
        159, 161, 162, 972, 165, 166, 167, 168, 144, 170, 171, 172, 173, 174,
        175, 176